# TP Forward Backward and FISTA


# Forward Backward and FISTA.

In this notebook we will code the Forward-Backward aLgorithm and its inertial an accelerated version FISTA to solve several optimization problems associated for example to the denoising and the inpaiting problem. I provide two images but you can use one of your own.

In [ ]:
import numpy as np
import scipy as scp
import pylab as pyl
import pywt
import pandas as pd
import holoviews as hv
import param
import panel as pn
import requests
import numpy.linalg as npl
import matplotlib.pyplot as plt

from panel.pane import LaTeX
hv.extension('bokeh')
import warnings
warnings.filterwarnings('ignore')
from PIL import Image
from io import BytesIO

In [ ]:
local=0
def chargeData(name):
    if local:
        if name=='Lenna':
            res=np.array(Image.open("./Archive/img/Lenna.jpg")).astype(float)
        if name=='Canaletto':
            res=np.array(Image.open("./Archive/img/Canaletto.jpeg")).astype(float)
        if name=='Minotaure':
            res=np.array(Image.open("./Archive/img/MinotaureBruite.jpeg")).astype(float)   
        if name=='Cartoon':
            res=np.array(Image.open("./Archive/img/Cartoon.jpg")).astype(float) 
    else:
        if name=='Lenna':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Lenna.jpg'        
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='Canaletto':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Canaletto.jpeg'
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='Minotaure':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/MinotaureBruite.jpeg'
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='Cartoon':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Cartoon.jpg'        
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
    return res

In [ ]:
im=chargeData('Canaletto')
im2=chargeData('Lenna')
imagesRef= {"Lenna" : im2,"Canaletto" : im}
options = dict(cmap='gray',xaxis=None,yaxis=None,width=400,height=400,toolbar=None)
pn.Row(hv.Image(im).opts(**options),hv.Image(im2).opts(**options))

In [ ]:
def PSNR(I,Iref):
    temp=I.ravel()
    tempref=Iref.ravel()
    NbP=I.size
    EQM=np.sum((temp-tempref)**2)/NbP
    b=np.max(np.abs(tempref))**2
    return 10*np.log10(b/EQM)

# Inpainting

For the inpainting problem, we suppose we have data $y=Mx^0+b$ which are images of a target image $x^0$ through a Masking operator $M$ and we will consider also that data may be corrupted by an additional noise $b$. We want to estimate $x^0$ solving the following optimization problem 
\begin{equation}\label{Eq1}
\frac{1}{2}\Vert Mx-y\Vert^2+\lambda \Vert Tx\Vert_1
\end{equation}
where $T$ is an orthogonal transformation such that $Tx^0$ is sparse. If $x^0$ is piecewise smooth, we can use an orthogonal wavelet transform such as Daubechies 2 or 4. To perform the FB algorithm, we need to compute the gradient of $\frac{1}{2}\Vert Mx-y\Vert^2$ and the proximal operator $\lambda \Vert Tx\Vert_1$. Some help is provioded further in the notebook.

If the noise $b$ vanishes, i.e $y=Mx^0$ where $M$ is the masking operator, it may be better to solve the following optimization problem :
\begin{equation}\label{Eq2}
\underset{x}{\min}\Vert Tx\Vert_1\text{ under the constraint }y=Mx  
\end{equation}
This problem can't be solved directly using the FB algorithm since it can't be stated as the sum of a differentiable function which gradient is Lipschitz and of a function which proximal operator is known.

Moreover, in any pratical problem, there is actually some noise, even it can be small.

It can be shown that solutions of \eqref{Eq1} converge to solutions of \eqref{Eq2} when $\lambda$ tends to 0. 
The advantage of \eqref{Eq1} is that it can be solved using FB.
Solving \eqref{Eq1} allows to inpaint and to regularize (somehow remove some noise). The regularization effect grows with $\lambda$. In many practical case, with small noise, $\lambda$ must be chosen quiete small in \eqref{Eq1}. 
The main drawback of such a choice is that it slows down the algorithm, an higher number of iterations is needed to get a good numerical result. When we use these methods, we must be aware that $$\textbf{the choice of $\lambda$ of the number of iterations and the starting point of this algorithm is essential}$$. If the numerical resultats are not as nice as you wish, you may have to change the value of $\lambda$ or increase the number of iterations.



# A few known results on explicit Gradient Descent and Forward-backward (FB)

If $F=f+g$ is a convex fucntion, sum of two convex functions, $f$ a differentiable function which gradient is $L-$Lipschitz and $g$ a convex function such that $prox_g$ is known, the Forward-Backward algorithm is defined by 
$$x_{n+1}=prox_{hg}(x_n-h\nabla f(x_n))=Tx_n\quad \text{ with }T:=prox_{hg}\circ (Id-h\nabla f)$$
One can show that the sequence $(F(x_n)-F(x^*))_{n\in\mathbb{N}}$ is non increasing and moreover 
$$F(x_n)-F(x^*)\leqslant \frac{2\Vert x_0-x^*\Vert^2}{hn}$$
This decay rate $\frac{1}{n}$ is optimal in the sens that it is impossible to get a bound decaying like $\frac{1}{n^{\delta}}$ with $\delta>1$ for all convex fucntions. Nevertheless it can be proved that if $h<\frac{1}{L}$ one actually has
$$F(x_n)-F(x^*)=o\left(\frac{1}{n}\right).$$  
If the function $f$ is strongly convex the decay is geometrical.

# Some rermarks on iteratives algorithms and FB in particular.

When you use an iterative algorithm depending on one or more parameters, you must be careful to evaluate the results, the outputs of the algorithm. It is crucial to check some points :

1) Check that the algorithm is close to converge... that is check that enough iterations have been done. This number of iterations may change depending on parameters and on the starting point $x_0$ chosen to build ythe sequence.

To check this point, I Invite you to display the curve of the values $F(x_n)$ and to think twice about what you doing and what you see on the outputs. For example, if you are performing inpainting and if some black pixels left, or if the sequence $F(x_n)$ is still decaying quite fast, it means that it may be relevant to increse the number of iterations. More generally, observing the artefacts of defects on the output may give some clues of what can be done to improve the result. Moreover, have a look to the sequence $F(x_n)$ may indicate that less iterations could have been performed to get a similar output.

2) Explore the possible values of paramters. It may occur that some values of parameters provide some relevant results and some don't. Before pretending that an algorithm is not efficient you must ensure that the parameters have been chosen wisely. These parameters can be the ones of the fucntion to minimize such as $\lambda$ in \eqref{Eq1} or the internal parameters of the algorithm such as the descent step for example. Making an Experiments Plan may be a good way to estimate the values of good parameters.

One can observe that the value of $F(x)$ is not the only criterion to evaluate the output of an algorithm, actually for different values of $\lambda$, the PSNR may be more relevant. We can't avoid thinking... 

3) When it it is possible choose a good starting point. Iterative algorithms build images (or signals) which converges to a minimiozer of the fucntion $F$. the choice of the staring point may be crucial. 
If we do not have any information, we can choose the constant image 0 or any relevant image, for example the masked image if we are performing inapainting or the noisy image if we are performing denoising. If for any problem a simple and quick method may provide a rough or innacurate output, it can be used as a starting point of the method. For the inpainting, we may start using a simple median filtering. The image is divided into small squares and the holes are filled up with the median value of the observed data.

One can often distinguish two steps during the optimization of the function $F$, a first one to go from the starting point to a good image, a coarse approximation of the whished output and a second one from the coarse approximation to the whished output. It may be difficult to avoid the second step but the first one may be shortened using a good starting point.

In many situations to compare several methods with various parameters or wavelet bases, it can be useful to compute the PSNR of each output on a benchmark to get a quite fair criterion for the comparison.


## Forward Backward for the inpainting.

We can start using observations without any noise and apply a mask. To create a 2D mask one can use the following commands : 

In [ ]:
np.random.seed(seed=1)
n1,n2=np.shape(im)
r=np.random.rand(n1,n2)
M=(r<0.8)
M = M*1.0

We can of course change the proportion of pixels that are masked.

We can mask the image by the following command :


In [ ]:
temp=M*im
pn.Row(hv.Image(im).opts(cmap='gray',width=400,height=400),hv.Image(temp).opts(cmap='gray',width=400,height=400))

<font color="red">

Le masquage a permis d'ajouter des bruits à l'image comme nous le voyons ci-dessus. Selon le seuil de coefficients à conserver, le masquage est plus ou moins fort. Plus la valeur du seuil est élevée, moins le masquage est fort.
    </font>

### Initilisation by median filter.

Implement and test a median filter which inpaint roughly the image replacing missing pixels using the median of values of observed pixels on vois*vois squares.

In [ ]:
def FiltreMedianB(im,masque,vois):
    MasIm=masque*im
    imrec=np.copy(MasIm)
    n1,n2=np.shape(im)
    K1=int(np.floor(n1/vois))
    K2=int(np.floor(n2/vois))
    for cel1 in range(vois):
        for cel2 in range(vois):
            for i in range(K1):
                for j in range(K2):
                    if np.abs(imrec[i+cel1*K1,j+cel2*K2])==0:
                        imrec[i+cel1*K1,j+cel2*K2]=np.median(imrec[cel1*K1:(cel1+1)*K1,cel2*K2:(cel2+1)*K2])
    return(imrec)

def FiltreMedian(im,masque,vois):
    imrec=np.copy(im)
    n1,n2=np.shape(im)
    #K1=int(np.floor(n1/vois))
    #K2=int(np.floor(n2/vois))
    im_mas=masque*im
    win=np.zeros(vois**2)
    for i in range(np.int(vois/2),n1-np.int(vois/2)):
        for j in range(np.int(vois/2),n2-np.int(vois/2)):
            if im_mas[i,j]==0:
                t=0
                for m in range(vois):
                    for n in range(vois):
                        if im_mas[i+m-np.int(vois/2)][j+n-np.int(vois/2)]!=0:
                            win[t]=im_mas[i+m-np.int(vois/2)][j+n-np.int(vois/2)]
                            t=t+1
                imrec[i,j]=np.median(win[0:t])       
    return(imrec)

In [ ]:
np.random.seed(seed=1)
n1,n2=np.shape(im)
r=np.random.rand(n1,n2)
M=(r<0.5)
M = M*1.0
irectest=FiltreMedian(im,M,7)
pn.Row(hv.Image(im).opts(title="Image Initiale",cmap='gray',width=400,height=400),hv.Image(temp).opts(title="Image Bruitée",cmap='gray',width=400,height=400),hv.Image(irectest).opts(title="Image Reconstruite",cmap='gray',width=400,height=400))

<font color="red">
En comparant les trois images, nous constatons que le fonction $FiltreMedian$ nous a permis de réduire considérablement le bruitage effectué sur l'image initiale.
    
</font>

In [ ]:
irectest=FiltreMedian(im,M,1)
irectest2=FiltreMedian(im,M,30)
pn.Row(pn.Column(hv.Image(im).opts(title="Image Initiale",cmap='gray',width=400,height=400),hv.Image(temp).opts(title="Image Bruitée",cmap='gray',width=400,height=400))
       ,pn.Column(hv.Image(irectest).opts(title="Image Reconstruite vois=1",cmap='gray',width=400,height=400),hv.Image(irectest2).opts(title="Image Reconstruite vois=60",cmap='gray',width=400,height=400)))

<font color="red">
    
Nous reprenons l'expérience précédente en faisant varier la valeur du paramètre $vois$ de la fonction. Nous constatons que pour de petites valeurs du paramètre, la qualité de l'image reconstruite n'est pas bonne. Elle est même mauvaise. Il en est de même pour de trop grandes valeurs de ce même paramètre où nous obtenons une image assez pixélisée. En effet, si le paramètre $vois$ est trop petit nous n'avons pas assez d'informations lorsque nous calculons la médiane des coefficients voisins. Alors que lorsque nous le prenons trop grand, nous prennons en compte trop d'informations et ainsi l'approximation est assez faussée. 
    </font>

### Back to FB

Implement a function computing the gradient of the function $f(x)=\frac{1}{2}\Vert M_{1}x-y\Vert^2$ :

<font color="red">
    
$\nabla f(x) = M_{1}^{T}(M_{1} x - y)$
    
Soit  $M_{1}:\mathbb{R}^{n\times m}\to \mathbb{R}^{n\times m} $ l'opérateur masque, on définit la matrice M de la taille $n \times m$:
    $M_{1} x=M \times x$
    
   Alors l'opérateur $M_{1}$ est une matrice diagonale de taille $(nm)\times (nm)$ avec les valeurs sous la diagonal sont 1 ou 0
    
 $\nabla f(x) = M_{1}^{T}(M_{1} x - y)=M_{1}^{T}(M\times x - y)=M\times(M\times x - y)$
   

In [ ]:
def GradientInpainting(x,b,M):
    g=M*(M*x-b)
    return g

For which value of $L$ this gradient is $L-$Lipschitz ?


<font color="red">
$\|\nabla f(a)-\nabla f(b)\| = \|M_{1}^{T}(M_{1} a - y)-M_{1}^{T}(M_{1} b - y)\|$
    
 $\|\nabla f(a)-\nabla f(b)\|=\|M_{1}^{T}(M_{1} (a-b)\| \leq \|M_{1}^{T}(M_{1})\| \|a-b\|$  
Le gradient est lipschitzien avec comme coefficient $L=|||M_1|||^2=1$.
    </font>

The following functions compute the proximal operator of the function $g(x)=\lambda \Vert Tx\Vert_1$ where $T$ is an orthogonal wavelet transform.

In [ ]:
def SeuillageDouxOndelettes(I,wave,Seuil):
    L=pywt.dwt_max_level(len(I),pywt.Wavelet(wave).dec_len)
    wavelet_coeffs= pywt.wavedecn(I, wave, mode='per', level=L)
    arr, coeff_slices, coeff_shapes = pywt.ravel_coeffs(wavelet_coeffs)
    temp=pywt.threshold(arr,Seuil,mode='soft')
    test=pywt.unravel_coeffs(temp, coeff_slices, coeff_shapes, output_format='wavedecn')
    Irec=pywt.waverecn(test, wave,mode='per')
    return Irec
def Normel1Ondelettes(I,wave):
    L=pywt.dwt_max_level(len(I),pywt.Wavelet(wave).dec_len)
    wavelet_coeffs= pywt.wavedecn(I, wave, mode='per', level=L)
    arr, coeff_slices, coeff_shapes = pywt.ravel_coeffs(wavelet_coeffs)
    norml1=sum(np.abs(arr))
    return norml1

In [ ]:
wave='haar'
imrec=SeuillageDouxOndelettes(im,wave,3)

Implement and test an inpainting program using the FB algorithm minimizing the function \eqref{Eq1}.
Don't forget to clip the data at the end of the program.

In [ ]:
def ForwardBackwardInpaintingv2(y,M,step,lam,Niter,wave):
    seuil=lam*step
    #z=0*y
    z=FiltreMedian(y,M,7)
    F=np.zeros(Niter)
    for iter in range(Niter):
        z=SeuillageDouxOndelettes(z-step*GradientInpainting(z,y,M),wave,seuil)
        F[iter]=0.5*npl.norm(M*z-y)**2+lam*Normel1Ondelettes(z,wave)
    return np.clip(z,0,255),F

In [ ]:
lam=5
Niter=200
wave='db2'
np.random.seed(seed=1)
n1,n2=np.shape(im)
r=np.random.rand(n1,n2)
M=(r<0.5)
M = M*1.0
B=np.random.randn(n1,n2)
y=M*im+5*B
y=np.clip(y,0,255)
step=0.9
Z,CF=ForwardBackwardInpaintingv2(y,M,step,lam,Niter,wave)

In [ ]:
pn.Row(hv.Image(im).opts(cmap='gray',width=400,height=400),hv.Image(Z).opts(cmap='gray',width=400,height=400))

In [ ]:
plt.plot(CF)
print(CF[-1])

<font color="red">
Nous constatons que l'algorithme est bien un algorithme de descente. En effet, la fonction coût décroit au cours du temps. Lorsqu'on observe l'image obtenue, nous pouvons constater que nous obtenons une image d'assez bonne qualité même si la valeur minimale de la fonction coût reste élevé. Afin de mieux visualiser les effets des paramètres sur l'image reconstruite, nous allons construire un dashboard et visualiser les effets.
    </font>

In [ ]:
wavelist = ['haar','db2','db3','db4','coif1','coif2','coif3']

Implement a dashboard allowing a numerical exploration of the previous function. We could observe that small small values of $\lambda$ produce better output. Theoreticaly, if the data are noiseless, it would be better to take $\lambda$ as small as possible. You can see it if you display the PSNR of the output when you compute a number of iterations that is large enough.

The drawback of such a choice is that it slows down extremely the algorithm. If data are noisy the optimal value of $\lambda$ is proportional to the standart deviation of the noise.

Hence, you may prefer a small value of $\lambda$ but nottoo small to ensure the convergence of the algorithm.


In [ ]:
class FBInpaint(param.Parameterized):
    wave = param.ObjectSelector(default="db2",objects=wavelist)
    image = param.ObjectSelector(default="Canaletto",objects=imagesRef.keys())
    Niter = param.Integer(10,bounds=(0,300))
    lam = param.Number(5,bounds=(1,30))
    step = param.Number(1,bounds=(0.005,4))
    masquage = param.Number(0.5,bounds=(0.1,1))
    sigma=param.Number(5,bounds=(0,10))
    def view(self):
        ima=imagesRef[self.image]
        n1,n2=np.shape(ima)
        r=np.random.rand(n1,n2)
        M=(r<self.masquage)
        M = M*1.0
        B=np.random.randn(n1,n2)
        y=M*ima+self.sigma*B
        Isol,CF=ForwardBackwardInpaintingv2(y,M,self.step,self.lam,self.Niter,self.wave)
        
        #calcul du PSNR
        p1 = PSNR(y,im)
        p2 = PSNR(Isol,im)
        
        #affichage
        strp1="%2.2f" % p1
        strp2="%2.2f" % p2
        te1='PSNR image masque = '
        te2='PSNR image inpaint = '
        TN1=hv.Text(0.5,0.3,te1+strp1).opts(xaxis=None,yaxis=None,toolbar=None)
        TN2=hv.Text(0.5,0.7,te2+strp2).opts(xaxis=None,yaxis=None)
        
        return pn.Row(pn.Column(hv.Image(y).opts(title="Image Bruitée",cmap='gray',width=400,height=400),hv.Image(Isol).opts(title="Image reconstruite",cmap='gray',width=400,height=400),plt.plot(CF)),TN1*TN2)

In [ ]:
fbinpaint= FBInpaint()
pn.Row(fbinpaint.param,fbinpaint.view)

<font color = red>
    
Tout d'abord, l'image reconstruit est acceptable quand la nombre d'itération est 50. Toutefois, nous pouvons augementer le nombre d'itérations pour améliorer le résultat. Quand le nombre d'itérations est environ 200, la fonction coût se stabilise et notre algorithme converge. 
    
Le paramètre $\lambda$ n'apporte pas de gros changements. En effet, la valeur PSNR augmente jusqu'à une valeur optimale autour de $10$ puis décroît légèrement même si ces valeurs restent très proches. 
    
Nous constatons également que plus la valeur du pas est grande, moins l'image reconstruite est de bonne qualité. Lorsque cette valeur dépasse $2$, l'image de très mauvaise qualité. En effet, l'algorithme FB converge pour de valeurs du pas inférieur à $\frac{2}{L}$ avec $L=|||M_1|||^2$. Vu comment la matrice $M_1$ est construite, nous devons avoir $L=1$. Ainsi l'algorithme converge pour des valeurs de  $step$ inférieures à $2$.
    
Nous trouvons aussi que  la base d'ondelette utilisée a peu d'impact sur l'image reconstruite. 
Nous voyons aussi que quand la valeur de masque est plus grande, l'opérateur masque retient plus de pixels donc l'image bruitée est plus nette. D'où, l'image débruitée est meilleure.
    
</font>    



# Accélération de Nesterov, FISTA.

Yurii Nesterov has proposed in the 80's several methods to accelerate the explicit Gradient Descent. We will focus on one of these, the one that is described in the lesson. If you have a look to research paper be aware that the 2 words "Nesterov accelerations" may have several meanings... using interpolation or extrapolation, be specific to strongly convex functions or not. it may apply to the explicit Gradient Descent (GD) or to the Forward Backward algorithm. Moreover we will not use the original parameter of Nesterov or FISTA but the ones we proposed in 2014 with Antonin Chambolle to ensure the convergence of iterates and speed up the convergence. 

the acceleration of the GD proposed but Nesterov in 19!4 and adapted to FB under the name of FISTA by Beck and Teboulle in 2009 is easy the apply and without any additional computational cost.

The idea is to apply the FB (or the GD for a single differntiable function) to a shifted point of $x_n$ with a step $h<\frac{1}{L}$. 
Be careful this bound is more restrictive than the one for the classical GD :
$$x_{n+1}=T(x_n+\alpha_n(x_n-x_{n-1}))$$ 
where the sequence $\alpha_n$ is chosen in a suitable way and $T$ is the FB operator. We say that FISTA is an inertial method because it uses an inertial term as a memory of the last descent direction.

Let's give some key points of this method :
1) The original choice of Nesterov for the sequence $\alpha_n$ is the following :
\begin{equation}
\alpha_n=\frac{t_n-1}{t_{n+1}}\text{ avec }t_1=1\text{ et }t_{n+1}=\frac{1+\sqrt{1+t_n^2}}{2}
\end{equation}
2) For this choice we have 
$$F(x_n)-F(x^*)\leqslant \frac{2\Vert x_0-x^*\Vert^2}{hn^2}$$
3) We can take the more simple choice $\alpha_n=\frac{n-1}{n+a-1}$ with $a>3$ (and that's what you will implement) and in this cas we have 
$$F(x_n)-F(x^*)\leqslant \frac{(a-1)^2\Vert x_0-x^*\Vert^2}{2h(n+a)^2}$$ and moreover 
$F(x_n)-F(x^*)=o\left(\frac{1}{n^2}\right)$ and the sequence $(x_n)_{n\geqslant 1}$ converges. 
It can be noticed, there are no inertia in the first step ($\alpha_1=0$) and thus $x_1=T(x_0)$. 
The inertia appears for the computation of $x_2$. The original choice of Nesterov is very close to the choice $\alpha=3$.

4) In the case of a composite fucntion ($F$ is a sum of a diffrentiable fucntion $f$ and a possibly non smooth fucntion $g$), this inertial algorithm is called FISTA (Fast Iterative Soft Shrinkage Thresholding Algorithm) because when $g$ is the $\ell_1$ norm, the proximal operator is a soft thresholding but the name FISTA is the name of this algorithm even if $g$ is not an $\ell_1$ norm.

5) Unlike FB, the sequence $(F(x_n)-F(x^*))_{n\geqslant 1}$ given by FISTA is not necessarily non increasing. The previous bounds on FB or FISTA are only ... bounds... but does not describe the real decay rate.
In practical cases you should see that FISTA is often faster than FB.


Implement an inpaiting algorithm minimizing the fucntion \eqref{Eq1} using FISTA.

In [ ]:
def FISTAInpainting(y,M,step,lam,Niter,wave,alpha):
    seuil=lam*step
    #z=0*y
    z=FiltreMedian(y,M,8)
    F=np.zeros(Niter)
    z_old=z
    for k in np.arange(0,Niter):
        a = (k-1)/(k+alpha-1)
        w = z + a*(z-z_old)
        grad = GradientInpainting(w,y,M)
        z_old = z
        z = SeuillageDouxOndelettes(w - step*grad,wave,seuil)
        F[k] =  lam*Normel1Ondelettes(z,wave)+1/2*np.linalg.norm(M*z-y)**2
      
        
    return np.clip(z,0,255),F

In [ ]:
lam=10
Niter=30
wave='db2'
np.random.seed(seed=1)
n1,n2=np.shape(im)
r=np.random.rand(n1,n2)
M=(r<0.5)
M = M*1.0
B=np.random.randn(n1,n2)
y=M*im+5*B
y=np.clip(y,0,255)
step=0.9
alpha=3.1
Z,CF=FISTAInpainting(y,M,step,lam,Niter,wave,alpha)

In [ ]:
pn.Row(hv.Image(im).opts(cmap='gray',width=400,height=400),hv.Image(Z).opts(cmap='gray',width=400,height=400))

In [ ]:
plt.plot(CF)

In [ ]:
class FISTAInpaint(param.Parameterized):
    wave = param.ObjectSelector(default="db2",objects=wavelist)
    image = param.ObjectSelector(default="Canaletto",objects=imagesRef.keys())
    Niter = param.Integer(20,bounds=(0,300))
    lam = param.Number(10,bounds=(1,30))
    step = param.Number(0.9,bounds=(0.05,4))
    masquage = param.Number(0.5,bounds=(0.1,1))
    alpha = param.Number(3.1,bounds=(3,10))
    sigma=param.Number(5,bounds=(0,20))
    def view(self):
        ima=imagesRef[self.image]
        n1,n2=np.shape(ima)
        r=np.random.rand(n1,n2)
        r2=np.random.rand(n1,n2)
        M=(r<self.masquage)
        M = M*1.0
        B=np.random.randn(n1,n2)
        
        y=M*ima+self.sigma*B
        y=np.clip(y,0,255)
        Isol,CF=FISTAInpainting(y,M,self.step,self.lam,self.Niter,self.wave,self.alpha)
        
       #calcul du PSNR
        p1 = PSNR(y,im)
        p2 = PSNR(Isol,im)
        
        #affichage
        strp1="%2.2f" % p1
        strp2="%2.2f" % p2
        te1='PSNR image masque = '
        te2='PSNR image inpaint = '
        TN1=hv.Text(0.5,0.3,te1+strp1).opts(xaxis=None,yaxis=None,toolbar=None)
        TN2=hv.Text(0.5,0.7,te2+strp2).opts(xaxis=None,yaxis=None)
        
        return pn.Row(pn.Column(hv.Image(Isol).opts(cmap='gray',width=400,height=400),plt.plot(CF)),TN1*TN2)

In [ ]:
fistainpaint= FISTAInpaint()
pn.Row(fistainpaint.param,fistainpaint.view)

<font color = red>
   
Nous voyons que l'algorithme converge très vite, seulement en plus 15 iterations. Les paramètres comme la $base d'ondelette$, $Lam$, $Step$, $Masquage$ conservent la même fonction comme l'algorithme Forward-Backward. Pour le paramètre $Alpha$, on trouve que la valeur optimale est autour de 3.
    
</font>


## To go further (optional )

As it has been done for denoising, one can use translated wavelets to improve the quality of the inpaiting. 
if you want to use this trick, you don't have to compute the inpainting from the beginning for each wavelet basis but use the previous outputs as smart starting points for FISTA or FB. This variation of the algorithm allows to smooth some artefacts due to the shape of the chosen wavelet.  

## Denoising using Total Variation norm.

If we know that the image we want to denoise has a small Total Variation norm ($\ell_1$ norm of the gradient), one can estimate the image minimizing a fucntion. be careful, the gradient is the one of the image not of a funtion applyed to an image. the gradient of an image is a vector field with two components, a vertical and a horizontal one which are computed using finite differences. I made a choice in the following programs for the discretization of the gradient and of the associated adjoint operator the negative divergence. If you ant to use another discretization of the gradient, you will have to modify the computation of the negative divergence. 

If we denote $y=x^0+b$ the observations where $x^0$ is the image to estimate and $b$ an additive noise, this fucntion will be
\begin{equation}\label{Primal}\tag{Primal}
\frac{1}{2}\Vert x-y\Vert^2+\lambda \Vert \nabla x\Vert_1
\end{equation}

Minimize such a fucntion is equivalent to compute the proximal operator of the function 

Minimiser une telle fonctionnelle revient à caluler l'opérateur proximal de la fonction $g(x)=\lambda \Vert \nabla x\Vert_1$ but it does not exist any formul for such an operator. It also means that we can't directly apply the FB using the quadratic term as a differentiable function.

However, it exists a way to use the FB algorithm to solve this problem : Duality.
It is not possible to give many details about this approach and I ask you to read your notes about duality and conjugate fucntions. The dualisation which uses the notion of Fenchel-Rockafellar conjugate associates to the previous optimization problem the follwiong one  
\begin{equation}\label{Dual}\tag{Dual}
\min_{p}\frac{1}{2}\Vert div(p)+y\Vert^2+\iota_{\mathcal{B}_{\infty,\lambda}}(p)
\end{equation}
where $p$ is a gradient field and $\iota_{\mathcal{B}_{\infty,\lambda}}$ is the function equal to 0 on the ball 
$\ell_{\infty}$ of radius $\lambda$ and $+\infty$ ouside and $div$ is the divergence operator. We have a divergence here because the 
adjoint of the gradient operator is the negative divergence ans a $\ell_{\infty}$ ball because it is the conjugate function of the $\ell_1$ norm.

If $p^*$ is a solution of the previous dual problem then $x^*=y+div(p^*)$ is the solution of the primal original problem. The advantage of the second problem is that it can be solved using FB or FISTA.


The following functions compute the discrete gradient and the associated discrete negative divergence.
As previously said, it is a choice of dsicretization, not the only one.

In [ ]:
def GradientHor(x):
    y=x-np.roll(x,1,axis=1)
    y[:,0]=0
    return y
def GradientVer(x):
    y=x-np.roll(x,1,axis=0)
    y[0,:]=0
    return y
def DivHor(x):
    N=len(x[0])
    y=x-np.roll(x,-1,axis=1)
    y[:,0]=-x[:,1]
    y[:,N-1]=x[:,N-1]
    return y
def DivVer(x):
    N=len(x)
    y=x-np.roll(x,-1,axis=0)
    y[0,:]=-x[1,:]
    y[N-1,:]=x[N-1,:]
    return y
def Gradient(x):
    y=[]
    y.append(GradientHor(x))
    y.append(GradientVer(x))
    return y
def Div(y):
    x=DivHor(y[0])+DivVer(y[1])
    return x

In [ ]:
def ProjGradBouleInf(g,l):
    gh=g[0]
    gv=g[1]
    temp=g
    p0=gh-(gh-l)*(gh>l)-(gh+l)*(gh<-l)
    p1=gv-(gv-l)*(gv>l)-(gv+l)*(gv<-l)
    temp[0]=p0
    temp[1]=p1
    return temp

Implement a Python function solving the dual problem using the FB algorithm. 

In [ ]:
def FBDenoisingTV(y,l,step,Niter):
    x=0*y
    p=Gradient(x)
    for k in range(0,Niter):
        
    return np.clip(x,0,255)

Test the program on a example on a noisy image with additional gaussian noise. Why can you observe an cartoon effect on the output image ? Change the values of parameters to check the effects of each of them on the output.
The stemp must be set above $1/4$ to satisfy the condition $h<\frac{2}{L}$.

Create the associated dashboard and test the algorithm on different images and parameters. The output is three images (original, noisy and solution) and 2 PSNR (of the noisy image and of the output). You should observe a cartoon effect for large values of $\lambda$.

In [ ]:
class DenoisingTVFB(param.Parameterized):
    Niter = param.Integer(100,bounds=(0,500))
    image = param.ObjectSelector(default="Canaletto",objects=imagesRef.keys())
    lam = param.Number(9,bounds=(1,50))
    step = param.Number(0.1,bounds=(0.1,2))
    Sigma = param.Number(17,bounds=(1,100))
    #alpha = param.Number(3,bounds=(1,10))
    def view(self):
        
        return 

In [ ]:
denoisingTVFB=DenoisingTVFB()
pn.Row(denoisingTVFB.param,denoisingTVFB.view)

# And Next ...

1) One can speed up the denoising intitiazing by a wavelet thresholding... one may compare the two approaches with or without translations using PSNR.
2) Apply FISTA.
3) Compute an Experiments Plan to compare the different choices of parameters. YHou must be careful here because the computational time is quite large for each experiements. You must have an idea of the total time of your Experiment Plan before starting.

## Going further (optional).

### Several defintions of the Total Variation (TV) Norm.

It exists several definitions of the TV norm . The one we are using here is the anisotropic TV norm, it also exists an isotropic one :

https://en.wikipedia.org/wiki/Total_variation_denoising

The use of the isotropic TV norm leads to artefacts that are more homogenious in all directions. 
The difference between the two approaches are particularly visible on the corners and the sides of objects.

To deal with an isotropic norm, you have to change the norme in \eqref{Primal}, and thus the definition of the 
$\ell_\infty$ ball in the dual problem is a bit different.

You must be aware that this difference exists and be clear in your mind about the norms you are using. 


### Denoising of colored images.

It is possible de denoise colored images, applying the previous algorithms on each chromatic chanel, but it is also possible to work directly on the colored image and define a global 3D TV norm on the three chanels, summing over the whole image, the norm of a gradient with ... 6 components. The goal of this approach is to avoid incoherence between the 3 chromatic chanels.

## Take away message.

If you implement a FB algorithm, nesterov accelerations are often efficient and it is always useful to test and even to test several choices of $a$. beginning with $a=3$. One can notive that, at least theoretically, the sequence   
generated $(F(x_n)-F(x^*))_{n\geqslant 1}$ generated by FB as a geometrical decay when $F$ is strongly convex, which is not the case for the studied Nesterov scheme. But it exists some accelerations that are dedicated to this specific situation. In practical cases, for a reasonnable number of iterations, these Nesterov accelerations are really competitive.